<a target="_blank" href="https://colab.research.google.com/github/bettercodepaul/nixtla_intro_workshop/blob/main/Introduction_to_Nixtlaverse.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Introduction to Forecasting with Nixtla's Nixtlaverse

This notebook walks you through the very basics of forecasting time series with Nixtla's Nixtlaverse.

## Install and import necessary libraries

We use [Polars](https://docs.pola.rs/) for data wrangling, [Plotly](https://plotly.com/python/plotly-express/) for visualizations and Nixtla's [StatsForecast](https://nixtlaverse.nixtla.io/statsforecast/index.html) for basic time series forecasting.

In [ ]:
pip -q install statsforecast polars plotly

In [ ]:
import polars as pl
import plotly.express as px
from statsforecast import StatsForecast
from datetime import date

## Initial Exploration of the data

The data for this walk through is simple monthly sales data from various countries.

In [ ]:
# load the dataset
df = pl.read_parquet("https://github.com/bettercodepaul/nixtla_intro_workshop/raw/refs/heads/main/retail_sales.parquet")
# show a sample of 5 rows
df.sample(5)

We can visualize the time series with Plotly.

In [ ]:
# This shows a line plot with the sales data for each country
# Plotly comes with useful interactive features: zoom, hover and trace isolation in the legend via click/double click
px.line(df, x="date", y="sales", color="country")

In [ ]:
# Too many overlapping lines?
# We can use a facet plot that creates a separate plot for each country
fig = px.line(df, x="date", y="sales", facet_col="country", facet_col_wrap=2)
fig.update_yaxes(matches=None) # This ensures that each subplot has its own y-axis scale
fig.update_layout(height=800) # Adjust the height to fit all subplots nicely
fig.show()

# Hands-on: Exploratory Data Dibbling

Explore the time series! What do you find interesting? Are there any obvious patterns? Are there any outliers? What could be the reasons?

In [ ]:
# you can either code here your own explorations or just use the interactive diagrams above

## Preparing the data for Nixtla

Nixtla follows a specific naming convention for the time series data.

- `unique_id`: an identifier to distinguish different time series in the same data set
- `ds`: the date or time column
- `y`: the actual value of the time series at that time

We rename the columns accordingly so that we don't have to specify the column names later on every method call.


In [ ]:
Y_df = df.rename(
    {
        "date": "ds",
        "sales": "y",
        "country": "unique_id",
    }
)
Y_df.sample(5)

In [ ]:
# Nixtla supports plotting time series using matplotlib, but we normally prefer Plotly for its interactive plots.
StatsForecast.plot(Y_df)

## Forecasting using Nixtla's StatsForecast

Nixtla's StatsForecast package comes with a lot of classic forecasting algorithms. We won't go into the details of the different algorithms in this workshop. If you would like to know more details about them we highly recommend the freely available book ["Forecasting: Principles and Practice, the Pythonic Way"](https://otexts.com/fpppy/) by Rob Hyndman.

As you might have noticed, the data contains a strong seasonal component in each year.
This is caused by different cultural habits, business practices and laws in each country.

Most algorithms that support to model this seasonal component have to be given the seasonal length. As we have a yearly seasonality and monthly data the seasonal length will be 12, which means that every 12 time steps the same period is repeated.

We will use 4 models in this example:

- [`HistoricAverage`](https://nixtlaverse.nixtla.io/statsforecast/src/core/models.html#historicaverage): forecast the mean of all past observations
- [`SeasonalNaive`](https://nixtlaverse.nixtla.io/statsforecast/src/core/models.html#seasonalnaive): forecast the last value of the same period (e.g. the same month of the previous year)
- [`HoltWinters`](https://nixtlaverse.nixtla.io/statsforecast/src/core/models.html#holtwinters): an exponential smoothing model, that models trend and seasonality
- [`AutoARIMA`](https://nixtlaverse.nixtla.io/statsforecast/src/core/models.html#autoarima): THE classic time series model, that models trend and seasonality. The parameters for the underlying SARIMA model are automatically selected.

In [ ]:
from statsforecast.models import (
    HistoricAverage,
    SeasonalNaive,
    HoltWinters,
    AutoARIMA,
)
models = [
    HistoricAverage(),
    SeasonalNaive(season_length=12),
    HoltWinters(season_length=12),
    AutoARIMA(season_length=12),
]

In [ ]:
# initialize the StatsForecast object with the models and frequency
sf = StatsForecast(
    models=models,
    freq="1mo",
)

In [ ]:
# create a forecast for the next 48 months (4 years) using all available data and all models
forecasts_df = sf.forecast(df=Y_df, h=48)
forecasts_df.sample(5)

We use Plotly to visualize the forecast.

In [ ]:
# append the forecasts to the original data
plot_df = pl.concat([Y_df, forecasts_df], how="diagonal")

# you could filter the data to only show a subset of countries
# plot_df = plot_df.filter(pl.col("unique_id").is_in(["Italien", "Japan"]))

# or you could select only a subset of the models
# plot_df = plot_df.select("unique_id", "ds", "y", "SeasonalNaive", "AutoARIMA")

y_columns = [c for c in plot_df.columns if c not in ["unique_id", "ds"]]
fig = px.line(plot_df, x="ds", y=y_columns, facet_col="unique_id", facet_col_wrap=2)
fig.update_yaxes(matches=None)
fig.update_layout(height=800)
fig.show()

In [ ]:
# the Nixtla equivalent to the code above is more concise, but less interactive
# sf.plot(Y_df, forecasts_df)

## Hands-on: Eyeballed Forecast Analysis

Explore the forecasts! What do you find interesting? Are the models capable of reproducing the seasonal patterns? Is there a model you would prefer over the others?

In [ ]:
# you can either code here your own explorations or just use the interactive diagrams above

Make another forecast, but now start at January 2018

In [ ]:
Y_df_before_2018 = Y_df.filter(pl.col("ds").dt.year().lt(2018))

In [ ]:
# use the code from above to create the forecast using Y_df_before_2018
# visualize the forecast using the code from above, but still us Y_df, so that you can compare the forecast with the actual values

## Forecast Validation

As demonstrated in the hands-on example, we can evaluate the accuracy of our model by comparing its forecasts for past periods (derived from "old data") to the actual values we already know today.

However, one of the most significant challenges in validating forecasts with this method is future leakage. It is crucial to ensure that the forecasts are not influenced by future data, as doing so would compromise the integrity of the evaluation. Unfortunately, in practice, completely avoiding future leakage is almost impossible. During the creation of the final forecasting system, many decisions must be made, and these decisions are often (unconsciously) influenced by information from the entire dataset.

Using a naive, random cross-validation scheme would definitely introduce future leakage. In such a scenario, the model would have access to data from the entire time period, allowing it to "learn" about future trends or events. This would lead to an overly optimistic cross-validation score that does not reflect the model's true performance on unseen data.

To mitigate this risk, we employ a specialized cross-validation scheme designed for time series data. This approach is often referred to as sliding window or expanding window validation. These techniques ensure a more robust evaluation of the model's predictive abilities across a wider range of temporal instances while maintaining critical properties:

1. the training data remains contiguous, adhering to the requirements of time series models, and
2. future data is systematically excluded from influencing the model during training.

By adhering to this method, we can arrive at a more realistic estimate of how well our model is likely to perform in practice.

In [ ]:
# visualization of an expanding window validation
from IPython.display import Image
Image(url="https://raw.githubusercontent.com/Nixtla/statsforecast/main/nbs/imgs/ChainedWindows.gif")

The `cross_validation` method from the `StatsForecast` class performs expanding window validation by default and accepts the following arguments:

- `df`: The training data frame containing the historical data.
- `h (int)`: The forecast horizon, representing the number of steps into the future to be forecasted. For example, in our case, this is set to 24 months.
- `step_size (int)`: The number of time steps by which the window is shifted for each validation iteration. In other words, this controls how far the window moves forward during the cross-validation process.
- `n_windows(int)`: The number of validation windows used for cross-validation. In other words, this determines how many historical forecasting processes you want to evaluate.

In [ ]:
# this takes some time...
cv_df = sf.cross_validation(
    df=Y_df,
    h=24,
    step_size=24, # try step_size 12 as well -> overlapping windows
    n_windows=4
)

In [ ]:
# you can check the resulting windows of the expanding window validation
windows = cv_df.group_by("cutoff").agg(pl.col("ds").max()).sort("cutoff")
windows

In [ ]:
colors = px.colors.qualitative.Plotly # used to color the windows

# create a line plot of the world wide monthly sales
fig = px.line(Y_df.group_by("ds").agg(pl.col("y").sum()).sort("ds"), x="ds", y="y")

# add the windows as vertical rectangles to the plot
for idx, window in enumerate(windows.rows()):
    start = window[0]
    end = window[1]
    fig.add_vrect(x0=start, x1=end, fillcolor=colors[idx%len(colors)], opacity=0.2)
    fig.add_annotation(x=window[0], xshift=5, xanchor="left", text=f"Window {idx+1}<BR>{start:%m/%y} - {end:%m/%y}", align="left", font=dict(color="grey"), showarrow=False)

fig

## Hands-on: Validation Judgement

Time series validation presents unique challenges because you often don’t have enough data to capture all relevant patterns.

- Seasonal models, for instance, require at least two periods of data (and almost all our data generating processes have a yearly seasonality!)
- Long-term trends evolve slowly, and in a start-up, the data may only reflect steady growth, making it nearly impossible for the model to predict a decline if it has never encountered one.
- Furthermore, rare events like pandemics or financial crises occur infrequently, perhaps once a decade, leaving little precedent to learn from.

Look at the data and the windows. Will this validation scheme be representative? Can we improve? What are the trade-offs?

In [ ]:
# Room for your thoughts

## Forecast Evaluation

There are several error metrics we can use to evaluate the models based on the forecast from the expanding window validation.

- **Mean Absolute Error**: [`mae`](https://nixtlaverse.nixtla.io/utilsforecast/losses.html#mean-absolute-error-mae)
    - measures the average forecasting accuracy using the absolute deviation
- **Root Mean Squared Error**: [`rmse`](https://nixtlaverse.nixtla.io/utilsforecast/losses.html#root-mean-squared-error)
    - measures the average forecasting accuracy using the squared deviation
- **Bias**: [`bias`](https://nixtlaverse.nixtla.io/utilsforecast/losses.html#bias)
    - measures the average forecasting bias using the deviation
- **Mean Absolute Percentage Error**: [`mape`](https://nixtlaverse.nixtla.io/utilsforecast/losses.html#mean-absolute-percentage-error)
    - measures the average forecasting accuracy using the absolute relative deviation
    - use this only for communication with stakeholders, not for choosing or comparing models

In [ ]:
from utilsforecast.losses import mae, rmse, bias, mape

In [ ]:
def evaluate_cv(df, metric):
    models = [c for c in df.columns if c not in ('unique_id', 'ds', 'cutoff', 'y')]
    evals = metric(cv_df, models=models)
    pos2model = dict(enumerate(models))
    return evals.with_columns(
        best_model=pl.concat_list(models).list.arg_min().replace_strict(pos2model)
    ).with_columns(pl.selectors.float().round(2))

In [ ]:
evaluation_df = evaluate_cv(cv_df, rmse)
evaluation_df.head()

## Hands-on: Forecast Evaluation

Try the different error metrics. Is it always the same model that "wins"?

Compare with a different validation scheme (different `n_windows` or `step_size`) 

In [ ]:
# Room for your analysis